# Wrythm – Data Extraction and Dataset

## Notebook Objective
This notebook focuses primarily on:

1. Extracting audio.
2. Extracting relevant musical features (MFCCs, BPM, rhythmic, melodic patterns, and key).
3. Building a consolidated dataset in JSON format for future use in genre classification.
4. Initial data exploration (EDA – exploratory data analysis).

---

## Pipeline 

### 1. Audio Collection
- 
- 

### 2. Feature Extraction
For each collected audio, extract:

- **MFCCs**: mean and variance to capture timbre.
- **BPM**: beats per minute (song tempo).
- **Rhythmic Cell**: onset histogram divided into bins.
- **Melodic Cell**: dominant notes per segment.
- **Key**: estimated musical key.
- **Duration**: audio duration in seconds.

Features will be stored together with metadata in JSON format.

### 3. Dataset Construction
- Consolidate JSON files into a single folder (`data/metadata/`).
- Ensure each record contains:
  - Track identifier (Spotify ID)
  - Track name and artists
  - Audio file path
  - Extracted features
  - Genre(s)

### 4. Initial Exploratory Analysis (EDA)
- Count of tracks per genre.
- Distribution of duration, BPM, and key.
- Basic statistics of MFCCs and rhythmic patterns.
- Check for consistency and possible missing data.


# Development


## 1 - Initial Setup: Imports and Preliminaries

## 2 - Data Collection & Feature Extraction


In [ ]:
import os
import re
import time
import json
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

# VARIÁVEIS DE CONTROLE
MAX_CLICKS = 1              # número máximo de "Load more" (None = ilimitado)
LOAD_WAIT_TIME = 3          # tempo de espera após clicar em "load more"
MIN_FILE_SIZE_KB = 10       # tamanho mínimo aceitável para considerar download válido
MAX_DOWNLOAD_RETRIES = 3    # tentativas para baixar cada música
MAX_SONGS_SCRAPE = 2        # limite de músicas para processar (None = ilimitado)
PARALLEL_DOWNLOADS = 5      

# CONFIGURAÇÕES
URL = "https://prosearch.tribeofnoise.com/search/index"
PLAYER_WAIT_TIME = 3  # tempo para player abrir após play/next

# Pastas de destino (No notebook de Samuel)
BASE_DIR = r"D:\Data_musicAI"
METADATA_DIR = os.path.join(BASE_DIR, "metadata")
PREVIEW_DIR = os.path.join(BASE_DIR, "preview")

os.makedirs(METADATA_DIR, exist_ok=True)
os.makedirs(PREVIEW_DIR, exist_ok=True)

logging.basicConfig(
    format='%(asctime)s %(levelname)s: %(message)s',
    level=logging.INFO,
    datefmt='%H:%M:%S'
)

# DRIVER
def iniciar_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

def wait_for(driver, by, selector, timeout=10):
    return WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((by, selector))
    )

# FUNÇÕES AUXILIARES DO SITE
def carregar_paginas(driver):
    limite = MAX_CLICKS if MAX_CLICKS is not None else 999999
    for i in range(limite):
        try:
            botao = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button#loadMore"))
            )
            botao.click()
            logging.info(f"[+] Página {i+2} carregada.")
            time.sleep(LOAD_WAIT_TIME)
        except TimeoutException:
            logging.info("[i] Não há mais páginas para carregar.")
            break

def sanitize_filename(name: str) -> str:
    """Remove caracteres inválidos para nomes de arquivos no Windows."""
    name = name.replace("\n", " ").replace("\r", " ")
    name = re.sub(r'[\\/:\*\?"<>\|]', "_", name)
    name = re.sub(r'\s+', " ", name).strip()
    return name

def ja_processada(titulo, artista):
    """Verifica se a música já foi salva em metadata (evita duplicatas)."""
    safe_title = sanitize_filename(titulo)
    safe_artist = sanitize_filename(artista)
    safe_name = f"{safe_artist} - {safe_title}"
    file_path = os.path.join(METADATA_DIR, f"{safe_name}.json")
    return os.path.exists(file_path)

def salvar_metadata(titulo, artista, link_download):
    """Salva os metadados de uma música como JSON único."""
    safe_title = sanitize_filename(titulo)
    safe_artist = sanitize_filename(artista)
    safe_name = f"{safe_artist} - {safe_title}"
    file_path = os.path.join(METADATA_DIR, f"{safe_name}.json")

    data = {
        "titulo": titulo,
        "artista": artista,
        "link_download": link_download
    }

    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

    logging.info(f"[💾] Metadata salva em: {file_path}")

def extrair_info_player(driver):
    """Extrai título, artista e link de download do player inferior."""
    # Extrair título e artista
    try:
        raw_text = driver.find_element(By.CSS_SELECTOR, ".c-player__song").text.strip()
        parts = raw_text.split("\n")
        titulo = parts[0].strip()
        artista = parts[1].strip() if len(parts) > 1 else "Artista não encontrado"
    except NoSuchElementException:
        titulo = "Título não encontrado"
        artista = "Artista não encontrado"

    link_download = None
    try:
        # Esperar até o botão de download do player estar clicável
        botao_download = WebDriverWait(driver, 8).until(
            EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "button.c-player__control-button.c-player__control-button--small[data-modal-toggle='#modal-download']")
            )
        )
        logging.info("[⬇] Botão de download encontrado.")
        botao_download.click()
        logging.info("[⬇] Botão de download clicado. Aguardando modal...")

        # Capturar link dentro do modal de download
        link_elem = WebDriverWait(driver, 8).until(
            EC.presence_of_element_located(
                (By.XPATH, "//div[@id='modal-download']//a[contains(@href,'/download/')]")
            )
        )
        link_download = link_elem.get_attribute("href")
        logging.info(f"Link de download capturado: {link_download}")

        # Fechar modal
        try:
            close_btn = driver.find_element(
                By.XPATH, "//div[@id='modal-download']//button[contains(@class,'c-modal__close')]"
            )
            close_btn.click()
            logging.info("Modal fechado.")
        except NoSuchElementException:
            logging.warning("Botão de fechar modal não encontrado (pode fechar sozinho).")

    except TimeoutException:
        logging.warning(" Timeout: botão ou modal de download não apareceu.")
    except NoSuchElementException:
        logging.warning("Elemento de download não encontrado.")
    except Exception as e:
        logging.warning(f"Erro ao capturar link de download: {e}")

    logging.info(f"[🎶] {titulo} — {artista}")
    if not link_download:
        logging.warning("Link de download não encontrado")

    return titulo, artista, link_download

def clicar_primeira_musica(driver):
    """Dá play na primeira música da lista para abrir o player inferior."""
    try:
        primeiro_botao = wait_for(driver, By.CSS_SELECTOR, "button.c-visual__button")
        primeiro_botao.click()
        logging.info("Primeira música iniciada (player aberto).")
        time.sleep(PLAYER_WAIT_TIME)
        return True
    except Exception as e:
        logging.error(f"Falha ao dar play na primeira música: {e}")
        return False

def proxima_musica(driver):
    """Clica no botão de próxima música no player inferior."""
    try:
        botao_proxima = driver.find_element(By.CSS_SELECTOR, "button.c-player__button--next")
        botao_proxima.click()
        logging.info("Próxima música.")
        time.sleep(PLAYER_WAIT_TIME)
        return True
    except NoSuchElementException:
        logging.warning("Botão de próxima música não encontrado.")
        return False
    except Exception as e:
        logging.warning(f"Não foi possível avançar para a próxima música: {e}")
        return False

# EXECUÇÃO DIRETA
driver = iniciar_driver()
driver.get(URL)
time.sleep(3)

carregar_paginas(driver)

if clicar_primeira_musica(driver):
    count = 0
    while True:
        if MAX_SONGS_SCRAPE is not None and count >= MAX_SONGS_SCRAPE:
            logging.info(f"Limite de {MAX_SONGS_SCRAPE} músicas atingido.")
            break

        titulo, artista, link = extrair_info_player(driver)

        if not ja_processada(titulo, artista):
            salvar_metadata(titulo, artista, link)
            count += 1
        else:
            logging.info(f"'{titulo}' de '{artista}' já foi processada. Pulando...")

        if not proxima_musica(driver):
            break

logging.info("[✓] Processo finalizado.")
driver.quit()


16:13:41 INFO: ====== WebDriver manager ======
16:13:42 INFO: Get LATEST chromedriver version for google-chrome
16:13:42 INFO: Get LATEST chromedriver version for google-chrome
16:13:43 INFO: Driver [C:\Users\Samue\.wdm\drivers\chromedriver\win64\141.0.7390.78\chromedriver-win32/chromedriver.exe] found in cache
16:14:02 INFO: [i] Não há mais páginas para carregar.
16:14:02 INFO: [▶] Primeira música iniciada (player aberto).
16:14:06 INFO: [⬇] Botão de download encontrado.
16:14:06 INFO: [⬇] Botão de download clicado. Aguardando modal...
16:14:14 WARNING: [!] Timeout: botão ou modal de download não apareceu.
16:14:14 INFO: [🎶] Golden — NeonNiteClub
16:14:14 WARNING: [!] Link de download não encontrado
16:14:14 INFO: [💾] Metadata salva em: D:\Data_musicAI\metadata\NeonNiteClub - Golden.json
16:14:14 WARNING: [!] Botão de próxima música não encontrado.
16:14:14 INFO: [✓] Processo finalizado.
